In [1]:
from pulp import *
from amplpy import AMPL, ampl_notebook

## Funções auxiliares
def print_solution(prob):
    print(f"Status: {LpStatus[prob.status]} --> {prob.objective.value()}")
    for v in prob.variables():
        print(f"{v.name}: {v.varValue}")

### Exercício 01



Uma cidade precisa decidir onde instalar bases do SAMU atendendo todas as regiões da cidade em até 15 minutos. Existem cinco possíveis locais de instalação. O local 1 atende as regiões sul e central em até 15 minutos. O local 2 atende as regiões sul, central e sudeste. O local 3 atende as regiões oeste e central em até 15 minutos. O local 4 atende as regiões central, norte e oeste em até 15 minutos. O local 5 atende as regiões oeste,  norte e sudeste em até 15 minutos.  Deseja-se minimizar o número de bases instaladas. Modele como um problema de programação linear inteira e resolva usando Python e/ou AMPL.

In [3]:
prob = LpProblem("Exercise_1", LpMinimize)

# Variável de decisão
x1 = LpVariable("Local1", cat=LpBinary)
x2 = LpVariable("Local2", cat=LpBinary)
x3 = LpVariable("Local3", cat=LpBinary)
x4 = LpVariable("Local4", cat=LpBinary)
x5 = LpVariable("Local5", cat=LpBinary)


# Função-objetivo 
prob += x1 + x2 + x3 + x4 + x5, "Instalação"

# Restrições
prob += x1 + x2 >=1, "Região Sul"
prob += x1 + x2 + x3 + x4 >=1, "Região Central"
prob += x2 + x5 >=1, "Região Sudeste"
prob += x3 + x4 + x5 >=1, "Região Oeste"
prob += x4 + x5 >=1, "Região Norte"

prob.solve()
print_solution(prob)

Status: Optimal --> 2.0
Local1: 1.0
Local2: 0.0
Local3: 0.0
Local4: 0.0
Local5: 1.0


### Exercício 02

O Problema da Instalação de Antenas: Uma empresa de telecomunicações deseja instalar antenas em alguns locais para cobrir seis áreas distintas. Foram identificados cinco locais possíveis para a instalação das antenas. Após algumas simulações, foi determinada a intensidade do sinal proveniente de uma antena colocada em cada local para cada área. A tabela a seguir resume esses níveis de intensidade:

| | Área 1 | Área 2 | Área 3 | Área 4 | Área 5 | Área 6 |
|-|--------|--------|--------|--------|--------|--------|
| Local A | 10 | 20 | 16 | 25 | 0 | 10 |
| Local B | 0 | 12 | 18 | 23 | 11 | 6 |
| Local C | 21 | 8 | 5 | 6 | 23 | 19 |
| Local D | 16 | 15 | 15 | 8 | 14 | 18 |
| Local E | 21 | 13 | 13 | 17 | 18 | 22 |


Os receptores reconhecem apenas sinais cujo nível seja pelo menos 18. Além disso, não é possível ter mais de um sinal atingindo o nível 18 na mesma área, pois isso causaria interferência. Finalmente, uma antena pode ser colocada no local E somente se uma antena também for instalada no local D (essa antena funcionaria como uma ponte). A empresa deseja determinar onde as antenas devem ser colocadas para cobrir o número máximo de áreas. Resolva usando Python e/ou AMPL.

In [55]:
# Inicializa o ambiente do AMPL com a licença gratuita e o módulo HiGHS
ampl_notebook(modules=["highs"], license_uuid="2e003210-ce0e-4ca9-bb4e-5d4a704b43a3")

# Inicializa o ambiente AMPL
ampl = AMPL()

# Define o modelo AMPL
modelo = """

param N{1..5, 1..6};

var x {i in 1..5} binary;
var y {j in 1..6} binary;

maximize MaxCobertura: sum {j in 1..6} y[j];

subject to CoberturaMinima{j in 1..5}:
    sum{i in 1..5: N[i,j] >= 18} x[i] >= y[j];

subject to Dependencia_E_D:
    x[5] <= x[4];  # Antena E só pode ser instalada se D também for

subject to Interferencia{j in 1..5}:
    sum{i in 1..5: N[i,j] >= 18} x[i] <= 1;
"""

# Carrega modelo e dados
ampl.eval(modelo)
ampl.read_data("./data/sinal.dat")

ampl.set_option('solver', 'highs')

# Resolve o problema
ampl.solve()

# Exibe os resultados
print("\nLocais com antena instalada:")
x_vals = ampl.get_variable("x").get_values()
for local, valor in dict(x_vals).items():
    if valor == 1:
        print(f"  Local {local}: antena instalada")

print("\nÁreas cobertas:")
y_vals = ampl.get_variable("y").get_values()
for area, valor in dict(y_vals).items():
    if valor == 1:
        print(f"  Área {area}: coberta")

# Objetivo
print("\nNúmero total de áreas cobertas:", ampl.get_objective("MaxCobertura").value())

Licensed to AMPL Academic Community Edition License for <mangussi.arthur@unifesp.br>.
HiGHS 1.10.0HiGHS 1.10.0: optimal solution; objective 5
0 simplex iterations
0 branching nodes

Locais com antena instalada:
  Local 1: antena instalada
  Local 3: antena instalada
  Local 4: antena instalada

Áreas cobertas:
  Área 1: coberta
  Área 2: coberta
  Área 4: coberta
  Área 5: coberta
  Área 6: coberta

Número total de áreas cobertas: 5.0


### Exercício 03

O Problema da Galeria de Arte: Considere uma galeria de arte com muitos corredores e curvas. A galeria está exibindo pinturas muito valiosas e você deseja mantê-las seguras. Você está planejando instalar câmeras de segurança em cada corredor para que as câmeras tenham todas as pinturas à vista. Se houver uma câmera de segurança em um corredor, ela poderá ver todas as pinturas do corredor. Se houver uma câmera no canto onde dois corredores se encontram (o turno), ela pode ver pinturas em ambos. Podemos modelar este sistema como um grafo onde os nós representam os lugares onde os corredores se encontram ou quando um corredor se torna um beco sem saída, e as arestas são os corredores. Considere o exemplo abaixo. Modele o problema como PPI e resolva usando Python e/ou AMPL.

In [ ]:
edges = [('A', 'B'), ('A', 'C'), ('B', 'D'), 
         ('C', 'D'), ('D', 'E'), ('E', 'F')]

vertices = set(v for edge in edges for v in edge)

prob = LpProblem("Galeria_de_Arte", LpMinimize)

x = {v: LpVariable(f'x_{v}', cat='Binary') for v in vertices}

prob += pulp.lpSum(x[v] for v in vertices)
for (u, v) in edges:
    prob += x[u] + x[v] >= 1

prob.solve()
print_solution(prob)

Status: Optimal --> 3.0
x_A: 0.0
x_B: 1.0
x_C: 1.0
x_D: 0.0
x_E: 1.0
x_F: 0.0
C
E
B


## Exercício 04

Uma empresa produz bobinas de papéis de cinco tamanhos diferentes a partir do corte de uma bobina-mestre. A empresa trabalha com 10 padrões de corte, ou seja, dez formas diferentes de cortar a bobina-mestre para obter seus produtos, conforme as colunas da matriz abaixo (cada coluna é um padrão de corte):

5 2 1 0 0 0 0 0 0 1 <br>
0 0 1 2 0 0 3 1 0 0 <br>
0 0 0 2 1 0 0 0 1 0 <br>
0 1 0 0 1 0 0 1 0 1 <br>
0 0 1 0 0 1 0 0 0 0 <br>


Por exemplo, a primeira coluna corresponde ao corte da bobina mestre em 5 bobinas do tipo 1. A segunda coluna corresponde ao corte da bobina mestre em duas bobinas do tipo 1 e uma bobina do tipo 4. E, assim por diante. A demanda desta semana é a seguinte: 18 bobinas do tipo 1, 31 bobinas do tipo 2, 25 bobinas do tipo 3, 15 bobinas do tipo 4 e 14 bobinas do tipo 5.

In [7]:
padroes = [
    [5,0,0,0,0],
    [2,0,0,1,0],
    [1,1,0,0,1],
    [0,2,2,0,0],
    [0,0,1,1,0],
    [0,0,0,0,1],
    [0,3,0,0,0],
    [0,1,0,1,0],
    [0,0,1,0,0],
    [1,0,0,1,0]
]

demanda = [18, 31, 25, 15, 14]

x = [LpVariable(f"x{i}", lowBound=0, cat='Integer') for i in range(10)]


prob = LpProblem("Bobinas_sem_estoque", LpMinimize)

prob += lpSum(x)

for i in range(5):
    prob += lpSum(padroes[j][i] * x[j] for j in range(10)) == demanda[i]

prob.solve()
print_solution(prob)

Status: Optimal --> 37.0
x0: 0.0
x1: 0.0
x2: 14.0
x3: 7.0
x4: 11.0
x5: 0.0
x6: 1.0
x7: 0.0
x8: 0.0
x9: 4.0


In [11]:
x = [LpVariable(f"x{i}", lowBound=0, cat='Integer') for i in range(10)]
y = [LpVariable(f"y{i}", cat=LpBinary) for i in range(10)]
M = 100000

prob = LpProblem("TempoTroca", LpMinimize)

prob += 0.25 * lpSum(x) + 3 * lpSum(y)

# Atendimento da Demanda
for i in range(5):
    prob += lpSum(padroes[j][i] * x[j] for j in range(10)) >= demanda[i]

for j in range(10):
    prob += x[j] <= M * y[j], f"Ativacao_y{j+1}"

prob.solve()
print("Status:", LpStatus[prob.status])
print("Tempo total mínimo:", value(prob.objective), "horas")
print("\nPadrões utilizados:")
for j in range(10):
    if x[j].value() > 0:
        print(f"Padrão {j+1}: {x[j].value()} bobinas (y{j+1} = {y[j].value()})")

Status: Optimal
Tempo total mínimo: 19.0 horas

Padrões utilizados:
Padrão 3: 18.0 bobinas (y3 = 1.0)
Padrão 4: 7.0 bobinas (y4 = 1.0)
Padrão 5: 15.0 bobinas (y5 = 1.0)


## Exercício 05

Fase 1: Um banco deve decidir quantos auditores serão necessários contratar em um horizonte de 6 meses de operação, março a agosto. As necessidades de esforço de auditoria são contabilizadas em termos de mão-de-obra de auditores experientes da seguinte forma:

| Mês    | Necessidade (Horas/mês) |
| ------ | ----------------------- |
| Março  | 7000                    |
| Abril  | 8000                    |
| Maio   | 10000                   |
| Junho  | 11000                   |
| Julho  | 11000                   |
| Agosto | 11000                   |

Cada auditor contratado como funcionário do banco, apesar de formado e aprovado em concurso, tem que ser treinado por um mês antes de poder atuar plenamente em sua função. Nesse treinamento, são utilizados auditores experientes do próprio banco que, deixando de concorrer na auditoria normal, dedicam 100 horas para cada auditor a ser treinado. Um auditor trabalha 150 horas por mês. Em 1º de fevereiro, o banco dispõe de 60 auditores experientes. O programa de contratação terá início em 1º de março.

Sabe-se também que o mercado de trabalho para auditores está muito instável, de forma que 10% da força de trabalho desses profissionais experientes deixa o banco a cada mês em busca de melhores salários. Um auditor experiente recebe do banco cerca de R$2000,00 por mês, enquanto o auditor em treinamento só recebe uma ajuda de R$150,00. Quando o número de auditores excede as necessidades, a carga de trabalho é reduzida, mas não são feitas demissões devido ao elevado custo do processo e ao risco de justiça. Quando isso acontece, novos auditores não são contratados e a evasão normal equilibra a força de trabalho.

Proponha uma solução objetivando minimizar os custos de operação do sistema de auditoria.

Fase 2: Utilizando o processo de terceirização
Paralelamente ao sistema de contratação formal para auditores existe a possibilidade de se obter mão-de-obra para auditorias via uma empresa de terceirização: a 3Part Consulting. Essa organização oferece auditores experientes (possivelmente egressos do sistema normal) e licenciados pela Câmara de Auditores Juramentados. Esse especialista custa R$ 2500,00 ao mês e pode ser retirado da folha a qualquer momento. Mesmo os que já trabalharam no sistema normal, a 3Part só exige a garantia mínima de um mês de trabalho para o profissional e que ele não trabalhe simultaneamente para o banco e para a 3Part. O banco pode ter sua licença auditada se descumprir esse requisito adicional. Reformular o problema levando em conta essa nova possibilidade.

In [3]:
horas = [7000, 8000, 10000, 11000, 7000, 11000]
meses = range(6)


prob = LpProblem("Auditoria_Bancaria", LpMinimize)

x = [LpVariable(f"x_{t}", lowBound=0, cat="Integer") 
for t in meses]
e = [LpVariable(f"e_{t}", lowBound=0) for t in meses]

prob += lpSum(2000 * e[t] + 150 * x[t] for t in meses)

for t in meses:
    prob += 150 * e[t] >= horas[t], f"Demanda_{t+1}"

prob += e[0] == 60, "Inicial"

for t in range(1, 6):
    prob += e[t] == 0.9 * (e[t-1] - (100/150) * x[t-1]) + x[t-1], f"Evolucao_{t+1}"

prob.solve()
print_solution(prob)

Status: Optimal --> 805470.0
e_0: 60.0
e_1: 54.0
e_2: 67.0
e_3: 73.5
e_4: 66.15
e_5: 73.535
x_0: 0.0
x_1: 46.0
x_2: 33.0
x_3: 0.0
x_4: 35.0
x_5: 0.0


In [13]:
from pulp import *

months = ["Mar", "Apr", "May", "Jun", "Jul", "Aug"]
demand = [7000, 8000, 10000, 11000, 11000, 11000]

prob = LpProblem("Auditoria", LpMinimize)

# Variáveis
x = {t: LpVariable(f"x_{t}", lowBound=0, cat='Integer') for t in months}
s = {t: LpVariable(f"s_{t}", lowBound=0) for t in months}
w = {t: LpVariable(f"w_{t}", lowBound=0) for t in months}
u = {t: LpVariable(f"u_{t}", lowBound=0, cat='Integer') for t in months}
z = {t: LpVariable(f"z_{t}", lowBound=0, cat='Integer') for t in months}

# Condição inicial
s_prev = 60

# Função objetivo
prob += lpSum(2000 * s[t] + 150 * u[t] + 2500 * z[t] for t in months)

for i, t in enumerate(months):
    prob += w[t] == s_prev + (u[months[i-1]] if i > 0 else 0)
    prob += s[t] == 0.9 * w[t]
    prob += 100 * x[t] <= 150 * s_prev
    prob += 150 * s[t] + 150 * z[t] >= demand[i]
    prob += u[t] == x[t]
    s_prev = s[t]

prob.solve()
print("Custo total:", value(prob.objective))
for t in months:
    print(f"{t}: Contratados={x[t].value()}, Experientes={s[t].value()}, Terceirizados={z[t].value()}")

Custo total: 799292.6
Mar: Contratados=6.0, Experientes=54.0, Terceirizados=0.0
Apr: Contratados=19.0, Experientes=54.0, Terceirizados=0.0
May: Contratados=16.0, Experientes=65.7, Terceirizados=1.0
Jun: Contratados=8.0, Experientes=73.53, Terceirizados=0.0
Jul: Contratados=7.0, Experientes=73.377, Terceirizados=0.0
Aug: Contratados=0.0, Experientes=72.3393, Terceirizados=1.0
